# De Novo Orthogonal Oligo Generator (DeNOOG)
author: Camillo Moschner | date: early 2022

## Import Statements

In [1]:
import random
import csv
import pandas as pd

import re
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import numpy as np
from collections import Counter
from tqdm.notebook import tqdm
from random import randint, choice
import networkx as nx

from nupack import *
from seqfold import dg, dg_cache, fold
from Bio.SeqUtils import MeltingTemp as mt
from Bio.Seq import Seq

In [2]:


from Levenshtein import distance # https://maxbachmann.github.io/Levenshtein/levenshtein.html#distance

import multiprocessing
from joblib import Parallel, delayed

from pylab import *
from ipywidgets import interact
from itertools import islice


## Function Definitions

In [3]:
def generate_random_DNA_sequence(length):
    """
        Generates a random sequence of DNA when given the length the sequence is supposed to be
        :param path: number
        :param: int
        :return: DNA sequence
        :return: str
    """
    seq, random_sequence = ['A', 'T', 'G', 'C'], []
    for i in range(0,length):
        x = choice(seq)
        random_sequence.append(str(x))
    random_sequence = ''.join(random_sequence)
    return random_sequence


def create_substitution_list(DNA_sequence,position):
    """
        Takes a DNA sequence string and generates a list of all DNA 
        substitutions possible at a SINGLE given position.
        :param1: str of DNA sequence
        :param2: int of sequence position that shall be substituted (starting with 0)
        :return: list of DNA sequences
    """
    DNA_sequence_list = list(DNA_sequence)
    seq = ['A', 'T', 'G', 'C']
    substitution_list = [DNA_sequence]
    for base_i in range(0,len(seq)):
        growing_list = ''
        DNA_sequence_list[position] = seq[base_i]
        for j in DNA_sequence_list:
            growing_list = growing_list + j
        if growing_list != DNA_sequence:
            substitution_list.append(growing_list)
    return substitution_list

def create_substitution_combinations(DNA_sequence,position_list):
    """
        Takes a DNA sequence string and generates a list of all DNA 
        substitutions possible at ALL given position.
        :param1: str of DNA sequence
        :param2: list of int of ALL sequence positions that shall  
                 be substituted (starting with 0)
        :return: list of DNA sequences
    """
    # intialise the first substitution list:
    entire_substitution_combination_list = create_substitution_list(DNA_sequence,position_list[0])
    # loop through every position to substitute, generating a list of all 
    # substitutions, add them to the list and start looping again:
    for i in range(1,len(position_list)):
        for j in range(0,len(entire_substitution_combination_list)):
            new_substitutions = create_substitution_list(entire_substitution_combination_list[j],position_list[i])
            new_substitutions.remove(entire_substitution_combination_list[j])
            entire_substitution_combination_list.extend(new_substitutions)     
    # Proof-check that the programme generated the expected amount of unique sequences:
    number_substitution_positions = len(position_list)
    number_expected_substitutions = 4 ** number_substitution_positions
    if number_expected_substitutions != len(set(entire_substitution_combination_list)):
        print('WARNING: {:} substitutions have been expected. \n{:} substitutions have been generated'.format(number_expected_substitutions,len(entire_substitution_combination_list)))
    # return a list of all possible combinations:
    return entire_substitution_combination_list



def list_to_CSV(list_x,CSV_name):
    """
        Takes a list and generates a CSV file with each 
        element of the list in a new indexed row.
        :param1: list
        :param2: str + '.csv'
        :return: CSV
    """
    with open(CSV_name, mode='w') as CSV_file:
        filewriter = csv.writer(CSV_file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        for i in range(0,len(list_x)):
            # index each row and append list element to column 1:
            new_row = [i+1, list_x[i]]
            filewriter.writerow(new_row)
    return 'CSV file \'{:}\' has been generated in the directory.'.format(CSV_name)

In [4]:
def get_complexes(seq, temp, return_dimer_conc = True):
    """Gives concentration of homo-intermolecular self-annealing oligos when starting with c=500nM of oligo
    """
    my_model = Model(material='dna', celsius=temp)
    a = Strand(seq, name='a')
    t1 = Tube(strands={a:5e-7}, complexes=SetSpec(max_size=2), name='')
    tube_result = tube_analysis(tubes=[t1], compute=['pairs', 'mfe'], model=my_model)
    if return_dimer_conc:
        for i, (my_complex, conc) in enumerate(tube_result.tubes[t1].complex_concentrations.items()):
            if i == 1: 
                return (conc), tube_result,tube_result["(a)"].mfe[0].energy
    return tube_result

def sec_struct_matrix_diagonal(test_seq):
    """Calculates the sum of diagonal probalities in the intramolecular self-annealing matrix
    """
    seq_array = test_seq['(a)'].pairs.to_array()
    #plt.imshow(seq_array)
    diag = 0
    for x in range(len(seq_array)):
        diag += seq_array[x,x]
    return diag

def reverse_compliment(query_seq):
    """Calculate reverse compliment of sequence in 5'->3' orientation
    """
    comp_dic, rev_compliment, reverse_seq = {'A':'T', 'T':'A', 'G':'C', 'C':'G'}, [], list(query_seq)[::-1]
    for rev_nt in reverse_seq:
        rev_compliment.append(comp_dic[rev_nt])
    return ''.join(rev_compliment)

def create_color_list(number_needed):
    color_list = []
    for i in range(number_needed):
        color_list.append('#%06X' % randint(0, 0xFFFFFF))
    return color_list

In [5]:
generate_random_DNA_sequence(1_000)

'CTGTGTGGTGTTTAAGGTAAGGCGTCTCATGGAAATGGATTGGTGGTCCGCCCTTGACCGCATTTGTGGCGACCGTTTCAACATAATTGCTTGGCTGAATCCTGAAGTATTCTCCAACCAGACGAGGGCACTGGTAAATTTCTATCTTCTCCCGCCATGCTTTTTGGTCGTGGTGATTCCGCGCTCTTGCGGACCCATATAACCCATCGCCTCAGAGCTCGAACTTGGTCCCGTGTTTGACCCTACAGTTCGCTTTTCGTGAGACAGGAATGGTTGCAAACTACTGTTCGTGCGAGAGAGCACAGATAAAACACATGCTGGTTCGTTCGACAAATCCCGGACGGTGGCCGTACATGGACGGATACCCACGGCTATACAACAAGCAGATCCAGTTCGTTACGGCAAAACTAGTCTGTAGTACTAGGGTGTGGCGAGTGAGAAAGCTTCGGATAACTTGCTCGAGGTATATAAAGAGCGGCGACAGCAAGACACCTGTAGGATGTAAGCATGTAAATGCAGCCCACAGGCGCTTAAGAGTATGGGCTGAACAAACTCGCGTAACCCAAGAAAAGGGATGAGCTCAATGCCACCCATGCTTCAACCTACGGGAGCCAAAAAAAAATCTGCGCTTACCGACCGCAACGCGTTCAAGTGGGGAGTCTCTTTATAGACCTCCAAGGGCACCCCCGTTACAATCTGGCTGCCTGCACTCCAATACAGGGGTTTTGGGTCTCCTGCTTCGAAGAGTTCACAAGAGTATGGTAGATGCGCGTATAGAACGAGTCGCCCCAATTCTTGAGTTCGCGATTATCTCAGACCGAACCGTCAAAGGAATCTGCTTTCGTGATCAGTCAAAACCCCATCGCGAATCATTCTTATAGAGGAAGCTCCCGAACGTAAGGAGAGGTTCGCTTGTGATGGCTTCCTAATAAGCGTCGAACGATCGTCTGGCTGCATTCATGTCACTACACCTCATGGCGAACAAGCGGCTTGGTGGCT

In [6]:
def primer_primer_dimerisation(seq_one, seq_two, temp):
    my_model = Model(material='dna', celsius=temp)
    a = Strand(seq_one, name='a')
    b = Strand(seq_two, name='b')
    t1 = Tube(strands={a:5e-7,b:5e-7}, complexes=SetSpec(max_size=2), name='')
    tube_result = tube_analysis(tubes=[t1], compute=['pairs', 'mfe'], model=my_model)
    for i, (my_complex, conc) in enumerate(tube_result.tubes[t1].complex_concentrations.items()):
        if i == 4:
            return conc

In [7]:
def generate_random_primer(length):
    """Generates a primer sequence de novo that fulfills a range of criteria
        :in1: int
        :in2: list
        :process: adds new str to in2
        :out1: str
    """
    seq = ['A', 'T', 'G', 'C']
    good_primer = ['no']
    iteration = 0
    while all( [x == 'yes' for x in good_primer] ) != True:
        iteration += 1
        good_primer = ['no']
        # create new random sequence:
        random_sequence = []
        for i in range(0,length):
            x = choice(seq)
            random_sequence.append(str(x))
        random_sequence = ''.join(random_sequence)
        # check new sequence:
        # 1- AGT in seq
        if 'ATG' not in random_sequence:
            good_primer[0] = 'yes'
        # 2- GC clamp
        end_3prime = random_sequence[-5:]
        GC_count_in_3prim_end = Counter(end_3prime)['G']+Counter(end_3prime)['C']
        if (GC_count_in_3prim_end == 3):
            good_primer.append('yes')
        else:
            good_primer.append('no')
        # 3- overall GC% between 50-58%
        GC_count_of_seq = Counter(random_sequence)['G']+Counter(random_sequence)['C']
        if (GC_count_of_seq/len(random_sequence) > 0.5) and (GC_count_of_seq/len(random_sequence) < 0.58):
            good_primer.append('yes')
        else:
            good_primer.append('no')
        if all( [x == 'yes' for x in good_primer] ) == True:
            # 4- exclude runs of 4+
            if (len(re.findall(r'A{4}',random_sequence)) > 0) or (len(re.findall(r'T{4}',random_sequence)) > 0) or (len(re.findall(r'G{4}',random_sequence)) > 0) or (len(re.findall(r'C{4}',random_sequence)) > 0):
                good_primer.append('no')
            else:
                good_primer.append('yes')
            if all( [x == 'yes' for x in good_primer] ) == True:
                # 5- include MFE (monomer ssDNA secondary structures) -> take only high values but not infinity
                mfe = dg(random_sequence)
                if (mfe > 4.) and (mfe < 5_000):
                    good_primer.append('yes')
                else:
                    good_primer.append('no')
                if all( [x == 'yes' for x in good_primer] ) == True:
                    # 6- exclude Type IIS RE rec_seqs
                    typeIISRE_rec_seqs = ['GGTCTC','GAGACC', #BsaI
                                          'CGTCTC','GAGACG', #BsmBI
                                          'GAAGAC','GTCTTC', #BpiI
                                          'GCTCTTC','GAAGAGC' #SapI
                                         ] 
                    for rec_seq in typeIISRE_rec_seqs:
                        if (len(re.findall(rec_seq,random_sequence)) > 0):
                            good_primer.append('no')
                        else:
                            good_primer.append('yes')
                    if all( [x == 'yes' for x in good_primer] ) == True:
                        # 7- self-annealing check (using NUPACK)
                        NUPACK_data = get_complexes(random_sequence,37)
                        if NUPACK_data[0] > 0.02:
                            good_primer.append('no')
                        else:
                            good_primer.append('yes')
                        if all( [x == 'yes' for x in good_primer] ) == True:
                            # 8- secondary structure/hairpin check2 (using NUPACK)
                            if sec_struct_matrix_diagonal(NUPACK_data[1]) < 19.5:
                                good_primer.append('no')
                            else:
                                good_primer.append('yes')
        #print(iteration, good_primer)
    print(iteration)
    return random_sequence

In [8]:
%%time
generate_random_primer(20)

130845
CPU times: user 18.3 s, sys: 194 ms, total: 18.5 s
Wall time: 18.4 s


'TTCTCTCCCTTTCCGCACTC'

----
## Execution

### 1. De novo oligo design

In [ ]:
num_cores = multiprocessing.cpu_count()

In [ ]:
if __name__ == "__main__":
    potential_primer_list_parallel = Parallel(n_jobs=num_cores)(delayed (generate_random_primer) (20)
                                                                for iterator in tqdm(range(12)) 
                                                               )

In [ ]:
potential_primer_list_parallel = pd.read_csv('master1000_DIY_primers.csv').primers

In [ ]:
len(potential_primer_list_parallel)

In [ ]:
potential_primer_df = pd.DataFrame({'primers' : potential_primer_list_parallel})
potential_primer_df['MFE'] = potential_primer_df.primers.apply(lambda p: dg(p))
potential_primer_df['Tm_GC'] = potential_primer_df.primers.apply(lambda p: mt.Tm_GC(Seq(p)))
potential_primer_df['Tm_NN'] = potential_primer_df.primers.apply(lambda p: mt.Tm_NN(Seq(p)))

In [ ]:
secondary_structure_complex_data_37 = [get_complexes(seq, 37) for seq in potential_primer_df.primers]
potential_primer_df['dimer_c_37'] = [item[0] for item in secondary_structure_complex_data_37]
potential_primer_df['dimer_c_72'] = potential_primer_df.primers.apply(lambda p: get_complexes(p, 72)[0])
potential_primer_df['dimer_c_98'] = potential_primer_df.primers.apply(lambda p: get_complexes(p, 98)[0])
potential_primer_df['secstruc_diag_37'] = [sec_struct_matrix_diagonal(item[1]) for item in secondary_structure_complex_data_37]

In [ ]:
potential_primer_df['GC_clamp'] = potential_primer_df.primers.apply(lambda seq: Counter(seq[-5:])['G']+Counter(seq[-5:])['C'])#.to_csv('master1000_DIY_primers.csv')#.primers.unique()
len(potential_primer_df.primers) == len(set(potential_primer_df.primers))

In [ ]:
potential_primer_df#.to_csv("master1000_DIY_primers.csv",index=False)  #.GC_clamp.describe()

In [ ]:
new_df = potential_primer_df.loc[(potential_primer_df.MFE>5) & 
                        (potential_primer_df.Tm_NN > 51) & 
                        (potential_primer_df.Tm_NN < 52)]
new_df

#### Primer-Primer Orthogonality 1: Primer Binding Site Competition -> Levenshtein Distance Matrix

In [ ]:
Levenshtein_list = []
for idx1, seq1 in enumerate(potential_primer_df.primers):
    for idx2, seq2 in enumerate(potential_primer_df.primers):
        Levenshtein_list.append(({seq1:seq2}))
Levenshtein_matrix_df = pd.DataFrame(np.array(Levenshtein_list).reshape(1_000,1_000), columns = list(potential_primer_df.primers))
Levenshtein_matrix_df['axis2'] = potential_primer_df.primers
Levenshtein_matrix_df.set_index('axis2',inplace=True)
for column_idx in range(len(Levenshtein_matrix_df.columns)):
    Levenshtein_matrix_df.iloc[:,column_idx] = (Levenshtein_matrix_df.iloc[:,column_idx].apply(lambda dictionary: list(dictionary.items())[0]))
    for index_idx in range(len(Levenshtein_matrix_df.index)):
        Levenshtein_matrix_df.iloc[index_idx,column_idx] = distance(Levenshtein_matrix_df.iloc[index_idx,column_idx][0], Levenshtein_matrix_df.iloc[index_idx,column_idx][1])

In [ ]:
Levenshtein_matrix_df#.iloc[:,0]#.describe().loc['top',:].max()

In [ ]:
dist = np.zeros((len(potential_primer_df.primers),len(potential_primer_df.primers)))
for i, seq1 in enumerate(potential_primer_df.primers):
    for j, seq2 in enumerate(potential_primer_df.primers):
        dist[i,j] = distance (seq1,seq2)

plt.imshow(dist)

In [ ]:
plt.hist(dist.flatten(),bins=20);

In [ ]:
sorted_primers_only = potential_primer_df.sort_values(by=['primers']).primers
dist_sorted = np.zeros((len(sorted_primers_only),len(sorted_primers_only)))
for i, seq1 in enumerate(sorted_primers_only):
    for j, seq2 in enumerate(sorted_primers_only):
        dist_sorted[i,j] = distance (seq1,seq2)
fig = plt.figure(figsize=(14,9))
plt.imshow(dist_sorted);

In [ ]:
dist_sorted

In [ ]:
test_distances = [15, 16, 17, 18]
plt.style.use('default')
fig = plt.figure(figsize=(14,9))
gs = gridspec.GridSpec(3,3,figure=fig)
ax_15, ax_16 = fig.add_subplot(gs[0,0]), fig.add_subplot(gs[0,1])
ax_15.set_title(f"15",size=10)
ax_15.imshow(dist_sorted==15)
ax_16.set_title(f"16",size=10)
ax_16.imshow(dist_sorted==16)

ax_17, ax_18 = fig.add_subplot(gs[1,0]), fig.add_subplot(gs[1,1])
ax_17.set_title(f"17",size=10)
ax_17.imshow(dist_sorted==17)
ax_18.set_title(f"18",size=10)
ax_18.imshow(dist_sorted==18)

ax_19, ax_20 = fig.add_subplot(gs[2,0]), fig.add_subplot(gs[2,1])
ax_19.set_title(f"19",size=10)
ax_19.imshow(dist_sorted==19)
ax_20.set_title(f"20",size=10)
ax_20.imshow(dist_sorted==20)
plt.show();

##### Network Approach Towards Cluster Identification

In [ ]:
"""First, choose the degree of dissimilarity between primer pairs
"""
all_pairs_twenty = np.where(dist==20)

twenty_pairs = []
for idx, column_value in enumerate(all_pairs_twenty[0]):
    column_seq = Levenshtein_matrix_df.columns[column_value]
    index_seq = Levenshtein_matrix_df.index[all_pairs_twenty[1][idx]]
    twenty_pairs.append((column_seq,index_seq))

In [ ]:
# total number of primer pairs with that step distance
len(twenty_pairs)
twenties_df = pd.DataFrame(twenty_pairs,columns=['seq1','seq2'])
twenties_df['seq1_counts'] = twenties_df.seq1.apply(lambda seq: Counter(seq))
twenties_df['seq2_counts'] = twenties_df.seq2.apply(lambda seq: Counter(seq))
twenties_df['seq1_count_length'] = twenties_df.seq1_counts.apply(lambda counts: len(counts))
twenties_df['seq2_count_length'] = twenties_df.seq2_counts.apply(lambda counts: len(counts))


In [ ]:
twenties_df.loc[(twenties_df.seq1_count_length == 4) & (twenties_df.seq2_count_length == 4)]

In [ ]:
subset = 200
network_edge_list_df = pd.DataFrame(twenty_pairs[:subset], columns=['seq1','seq2'])
network_edge_list_df['distance'] = [distance(x[0],x[1]) for x in twenty_pairs[:subset]]

In [ ]:
"""Create first network graph of all primer pairs
"""
G = nx.MultiDiGraph()
for row_idx in range(len(network_edge_list_df)):
    G.add_edge(network_edge_list_df.iloc[row_idx,:].seq1,network_edge_list_df.iloc[row_idx,:].seq2,arrowstyle="-") 

In [ ]:
"""Update first primer with its degree information
"""
network_edge_list_df['seq1_degrees'] = network_edge_list_df.seq1.apply(lambda seq: G.degree()[seq])
network_edge_list_df['seq2_degrees'] = network_edge_list_df.seq2.apply(lambda seq: G.degree()[seq])

In [ ]:
from itertools import product, combinations

In [ ]:
TAGAAAGGAGAGGGCGGGAT_combos = pd.DataFrame(list(combinations(Levenshtein_matrix_df['TAGAAAGGAGAGGGCGGGAT'].loc[
    Levenshtein_matrix_df['TAGAAAGGAGAGGGCGGGAT']>18].index,2)),columns=['seq1','seq2'])

In [ ]:
TAGAAAGGAGAGGGCGGGAT_combos['distance'] = list(map(lambda duo: Levenshtein_matrix_df.loc[duo[0],duo[1]],list(zip(TAGAAAGGAGAGGGCGGGAT_combos.seq1,TAGAAAGGAGAGGGCGGGAT_combos.seq2))))

In [ ]:
TAGAAAGGAGAGGGCGGGAT_combos.nlargest(3,'distance')

In [ ]:
combos = pd.DataFrame(list(combinations(['TAGAAAGGAGAGGGCGGGAT', 'ATAGGAGAGGAGGGAAAGGG', 'GCTCTCCTCCAACTAACACC', 'CCCGCTTCTTTCCTTCTCTG'],2)),columns=['seq1','seq2'])
combos['distance'] = list(map(lambda duo: Levenshtein_matrix_df.loc[duo[0],duo[1]],list(zip(combos.seq1,combos.seq2))))
combos['dimerisation_c'] = list(map(lambda duo: primer_primer_dimerisation(duo[0],duo[1],37),list(zip(combos.seq1,combos.seq2))))

In [ ]:
combos

In [ ]:
distance('AGC','GTT'), distance('TAC','GTT'), distance('AGC','TAC')

In [ ]:
random

In [ ]:
dist_selected_primers_df = pd.DataFrame({'primers' : pd.concat([network_edge_list_df.seq1, network_edge_list_df.seq2]).unique()})
dist_selected_primers_df['degrees'] = dist_selected_primers_df.primers.apply(lambda seq: G.degree()[seq])

In [ ]:
Levenshtein_matrix_df.loc[dist_selected_primers_df.loc[dist_selected_primers_df.degrees > 4].
                          primers,dist_selected_primers_df.loc[dist_selected_primers_df.degrees > 4].primers] #).astype(int))

In [ ]:
hits = []
for primer_seq in dist_selected_primers_df.primers:
    #print(primer_seq)
    current_primer_analysis_df = pd.concat([network_edge_list_df.loc[(network_edge_list_df.seq1 == primer_seq)], 
                                            network_edge_list_df.loc[(network_edge_list_df.seq2 == primer_seq)]
                                           ])
    if (current_primer_analysis_df.seq1_degrees.mean() > 5) and current_primer_analysis_df.seq2_degrees.mean() > 5:
        hits.append(current_primer_analysis_df)

In [ ]:
@interact
def show_hits(idx = (0,len(hits)-1)):
    display(hits[idx])

In [ ]:
Levenshtein_matrix_df.loc[Levenshtein_matrix_df.loc['CTACCCAACCAACACCTCCT']==20]

In [ ]:
dist_selected_primers_df.loc[dist_selected_primers_df.degrees > 10].primers.to_list()

In [ ]:
Levenshtein_matrix_df.loc[dist_selected_primers_df.loc[dist_selected_primers_df.degrees > 4].primers.to_list()].sort_values(by=['axis2']).to_a

In [ ]:
node_degree_df = pd.DataFrame(G.degree(), columns=['node','degree'])

In [ ]:
# get color list based on specific colormap
cmap = cm.get_cmap('magma_r', node_degree_df.degree.max())    # PiYG
color_dic = {}
for i in range(cmap.N):
    rgba = cmap(i)
    color_dic[i+1] = matplotlib.colors.rgb2hex(rgba)
f"{len(color_dic)} colors for max. {len(color_dic)} connections"

In [ ]:
color_map = []
for node in G.degree():
    color_map.append(color_dic[node[1]])

In [ ]:
options = {
    "font_size": 2,
    "node_size": 3000,
    #"node_color": "white",
    #"edgecolors": "black",
    "linewidths": 5,
    "width": 15,
}
plt.figure(figsize=(20,20))
nx.draw_networkx(G,node_size = 800, node_color=color_map, with_labels = True)

# Set margins for the axes so that nodes aren't clipped
ax = plt.gca()
ax.margins(0.20)
plt.axis("off")
plt.show()

#### Primer-Primer Orthogonality 2: Primer-to-Primer Annealing/Dimerisation -> NUPACK Matrix

In [ ]:
primer_primer_dimerisation('AGAGUAGGUGGGAGAGGGAA',
                           'CCUUCUCACACCCUUUCCUC', 37)

In [ ]:
my_model = Model(material='dna', celsius=37)
a = Strand("AGAGUAGGUGGGAGAGGGAA", name='a')
b = Strand("CCTTCTCACACCCTTTCCTC", name='b')
t1 = Tube(strands={a:5e-7,b:5e-7}, complexes=SetSpec(max_size=2), name='')
tube_result = tube_analysis(tubes=[t1], compute=['pairs', 'mfe'], model=my_model)
for i, (my_complex, conc) in enumerate(tube_result.tubes[t1].complex_concentrations.items()):
    if i == 4:
        print(my_complex, conc)

#### Guide Tree for the 1_000 Final Sequences

In [ ]:
f = open('master1000_DIY_primers_guide_tree.txt','r')

In [ ]:
with open('master1000_DIY_primers_guide_tree.txt', 'r') as file:
    data = file.read().replace('\n', '')

##### Save de novo primer

In [ ]:
fasta_string = []
for primer_idx, seq in enumerate(new_df.primers):
    fasta_string.append(f">primer_{primer_idx}\n{seq}\n")
fasta_string = ''.join(fasta_string)

In [ ]:
text_file = open("diy_primers.fasta", "w")
n = text_file.write(fasta_string)
text_file.close()

In [ ]:
potential_primer_df.to_csv('diy_primers.csv')

---
### 2. Template-based oligo design

#### 2.1 Homology regions

In [ ]:
def test_for_opt_primer(primer_list):
    """Generates a primer sequence de novo that fulfills a range of criteria
        :in1: int
        :in2: list
        :process: adds new str to in2
        :out1: str
    """
    seq = ['A', 'T', 'G', 'C']
    
    iteration = 0
    potential_primers = []
    for random_sequence in primer_list:
        iteration +=1
        good_primer = ['no']
        # check new sequence:
        # 1- AGT in seq
        #if 'ATG' not in random_sequence:
         #   good_primer[0] = 'yes'
        # 2- GC clamp
        end_3prime = random_sequence[-5:]
        GC_count_in_3prim_end = Counter(end_3prime)['G']+Counter(end_3prime)['C']
        if (GC_count_in_3prim_end >1) & (GC_count_in_3prim_end < 4):
            good_primer[0] = 'yes'
        else:
            good_primer.append('no')
         # 3- overall GC% between 50-58%
        GC_count_of_seq = Counter(random_sequence)['G']+Counter(random_sequence)['C']
        if (GC_count_of_seq/len(random_sequence) > 0.45) and (GC_count_of_seq/len(random_sequence) < 0.58):
            good_primer.append('yes')
        else:
            good_primer.append('no')
        # 4- exclude runs of 4+
        if (len(re.findall(r'A{4}',random_sequence)) > 0) or (len(re.findall(r'T{4}',random_sequence)) > 0) or (len(re.findall(r'G{4}',random_sequence)) > 0) or (len(re.findall(r'C{4}',random_sequence)) > 0):
            good_primer.append('no')
        else:
            good_primer.append('yes')
        if all( [x == 'yes' for x in good_primer] ) == True:
            # 5- include MFE (monomer ssDNA secondary structures) -> take only high values but not infinity
            mfe = dg(random_sequence)
            if (mfe > 0.5) and (mfe < 5_000):
                good_primer.append('yes')
            else:
                good_primer.append('no')
#                 if all( x == 'yes' for x in good_primer ) == True:
#                     # 6- exclude Type IIS RE rec_seqs
#                     typeIISRE_rec_seqs = ['GGTCTC','GAGACC', #BsaI
#                                           'CGTCTC','GAGACG', #BsmBI
#                                           'GAAGAC','GTCTTC', #BpiI
#                                           'GCTCTTC','GAAGAGC' #SapI
#                                          ] 
#                     for rec_seq in typeIISRE_rec_seqs:
#                         if (len(re.findall(rec_seq,random_sequence)) > 0):
#                             good_primer.append('no')
#                         else:
#                             good_primer.append('yes')
#             # 7- self-annealing check (using NUPACK)
#             NUPACK_data = get_complexes(random_sequence,37)
#             if NUPACK_data[0] > 0.02:
#                 good_primer.append('no')
#             else:
#                 good_primer.append('yes')
#             if all( x == 'yes' for x in good_primer ) == True:
#                 # 8- secondary structure/hairpin check2 (using NUPACK)
#                 if sec_struct_matrix_diagonal(NUPACK_data[1]) < 19.5:
#                     good_primer.append('no')
#                 else:
#                     good_primer.append('yes')
        #print(iteration,good_primer)
        if all( [x == 'yes' for x in good_primer ]) == True:
            potential_primers.append(random_sequence)
            #print(iteration,good_primer)
    print(iteration, len(potential_primers))
    return potential_primers

In [ ]:
'atggtagcccgtaaaggcgaagagctgttcactggtgtcgtccctattctggtggaactggatggtgatgtcaacggtcataagttttccgtgcgtggcgagggtgaaggtgacgcaactaatggtaaactgacgctgaagttcatctgtactactggtaaactgccggtaccttggccgactctggtaacgacgctgacttatggtgttcagtgctttgctcgttatccggaccatatgaagcagcatgacttcttcaagtccgccatgccggaaggctatgtgcaggaacgcacgatttcctttaaggatgacggcacgtacaaaacgcgtgcggaagtgaaatttgaaggcgataccctggtaaaccgcattgagctgaaaggcattgactttaaagaagatggcaatatcctgggccataagctggaatacaattttaacagccacaatgtttacatcaccgccgataaacaaaaaaatggcattaaagcgaattttaaaattcgccacaacgtggaggatggcagcgtgcagctggctgatcactaccagcaaaacactccaatcggtgatggtcctgttctgctgccagacaatcactatctgagcacgcaaagcgttctgtctaaagatccgaacgagaaacgcgatcatatggttctgctggagttcgtaaccgcagcgggcatcacgcatggtatggatgaactgtacaaaggttcgtaa'[:310]

In [ ]:
HL_primer_region = 'atggtagcccgtaaaggcgaagagctgttcactggtgtcgtccctattctggtggaactggatggtgatgtcaacggtcataagttttccgtgcgtggcgagggtgaaggtgacgcaactaatggtaaactgacgctgaagttcatctgtactactggtaaactgccggtaccttggccgactctggtaacgacgctgacttatggtgttcagtgctttgctcgttatccggaccatatgaagcagcatgacttcttcaagtccgccatgccggaaggctatgtgcaggaacgcacgatttcctttaaggatgacggcacgtacaaaacgcgtgcggaagtgaaatttgaaggcgataccctggtaaaccgcattgagctgaaaggcattgactttaaagaagatggcaatatcctgggccataagctggaatacaattttaacagccacaatgtttacatcaccgccgataaacaaaaaaatggcattaaagcgaattttaaaattcgccacaacgtggaggatggcagcgtgcagctggctgatcactaccagcaaaacactccaatcggtgatggtcctgttctgctgccagacaatcactatctgagcacgcaaagcgttctgtctaaagatccgaacgagaaacgcgatcatatggttctgctggagttcgtaaccgcagcgggcatcacgcatggtatggatgaactgtacaaaggttcgtaa'[:310]
HR_primer_region = 'CGGAAAACTTACCCTTAAATTGATTTGCACTACTGGAAAACTACCTGTTCCATGGCCAACACTTGTCACTACTTTGGGTTATGGTGTTCAATGCTTTGCGAGATACCCAGATCATATGAAACAGCATGACTTTTTCAAGAGTGCCATGCCCGAAGGTTATGTACAGGAAAGAACTATATTTTTCAAAGATGACGGGAACTACAAGACACGTGCTGAAGTCAAGTTTGAAGGTGATACCCTTGTTAATAGAATCGAGTTAAAAGGTATTGATTTTAAAGAAGATGGAAACATTCTTGGACACAAATTGGAATACAACTATAACTCACACAATGTATACATCACGGCAGACAAACAAAAGAATGGAATCAAAGCGAACTTCAAAATTAGACACAACATTGAAGATGGAGGTGTTCAACTAGCAGACCATTATCAACAAAATACTCCAATTGGCGATGGCCCTGTCCTTTTACCAGACAACCATTACCTGTCCTACCAATCTAAGCTTTCGAAAGATCCCAACGAAAAGAGAGAtCACATGGTCCTTCTTGAGTTTGTA'

In [ ]:
def slice_into_chunks(seq_l, chunk_length):
    """
    """
    return [seq_l[i:i+chunk_length] for i in range(0, len(seq_l), chunk_length)]

def window(seq, n=2):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result    
    for elem in it:
        result = result[1:] + (elem,)
        yield result

def moving_window(seq_l, window_length):
    return [''.join(x) for x in list(window(seq_l,n=window_length))]

In [ ]:
possible_HL_primers = moving_window(HL_primer_region,21)
possible_HR_primers = moving_window(HR_primer_region,23)
#possible_HL_primers.append('CCGTCCCTCTTCTTTCCCTT')
#possible_HL_primers.append('ATAGGAGAGGAGGGAAAGGG')
len(possible_HL_primers), len(possible_HR_primers)

In [ ]:
possible_HR_primers

In [ ]:
test_for_opt_primer(possible_HR_primers)
#possible_HL_primers

In [ ]:
fasta_string = []
for primer_idx, seq in enumerate(test_for_opt_primer(possible_HR_primers)):
    fasta_string.append(f">primer_{primer_idx}\n{seq}\n")
fasta_string = ''.join(fasta_string)
text_file = open("primer_search_mVenus.fasta", "w")
n = text_file.write(fasta_string)
text_file.close()

In [ ]:
distance('CAGTGGAAGGATGATGTCGTGCT','CCATCATTGCGGTCATAAAGTCG')

#### 2.2 Find good qPCR Primers

In [ ]:
# State overall DNA sequence in which you want primers should bind:

sfGFP_cds = 'TTATATGTTCAAGCCACGATGTTGCAGCATCGGCATAATCTTAGGTGCCTTACCGCGCCATTGTCGATACAGGCGTTCCAGATCTTCGCTGTTACCTCTGGAAAGGATCGCCTCGCGAAAACGCAGCCCATTTTCACGCGTTAATCCGCCCTGCTCAACAAACCACTGATAACCATCATCGGCCAACATTTGCGTCCACAGATAAGCGTAATAACCTGCAGCATATCCGCCACCAAAAATATGGGCGAAATAACTGCTGCGATAGCGTGGCGGTATAGCAGGAAGATCCATATTTTCCGCCACCAGCGCCCGCAATTCAAAATCATCGACATCCTGCATTGCTTCGTTTTCTTCCAGGCAATGCCAGCGCATATCGAGAAGTGCGGCGCTAAGCAGTTCGCTCATCTCATACCCTTTGTTGAACAGGCTGGCATTACGCATTTTCTGTTGCAGTTCGTCAGGCATTGCTGCCCCGCTCTGATAATGCCGGGCGTAGCGAGCGAATACCTGCGGATGCGTTGCCCAGTGTTCGTTGATTTGCGACGGAAATTCGACAAAATCACGCGGCGTGTTGGTGCCGGAAAGCGTGGCATAACGCTGGCGGGCAAAAAGGCCGTGCAGCGTATGACCAAATTCATGGAATAAGGTTATGACATCATCCCAGAGTAACAACGCAGGCTCACCGGCAGCGGGTTTCTGATAATTGCAGACGTTATAAATTACCGGATGTGTTTTATTAAGCGTTGATTGCTCAACAAAATTGCCCATCCATGCACCGCCGCTTTTTGAATCACGGGCGAAGAAATCACCGTAAAATAACGCCAGCCCCACGCCATTATGATCAAAAATTTCCCACACACGAACGTCAGGATGGTAGACAGGAATATCAAAACGTTCGACAAACTTAATACCGAAGAGCTGATTCGCGGTCCAGAATACACCTTCATTTAACACCGTGTTTAATTCAAAATATGGCTTGAGCTGCGCCTCATCAAGATCAAATTTCTCCCGCCGTACCTGTTCCGCATAAAATGCCCAGTCCCACGGCTGCGCGCTAAACCCGCCCTGCTGCTTATCGATAACCGCCTGTATGGAGGCTAATTCATCGCTCGCACGTTGACGCGCCGCTGGAACAATTTCCCGCATAAAGTTAAGTGCTGCTTCAGGTGTTTTTGCCATCTGATCGGCGATTTTCCATGCGGCATAATGAGGAAAACCAAGTAGTGTTGCCTGTTGTGCACGGATCTCCACCAGACGTTGAATGATAGCGCGGGTATCATTGGCATCATTTTTTTCCGCTCGCGTCCAGCCCGCAATAAACAGTTTTTCACGCGTCGCACGATCGCGCATTTCGGCAAGCGCCGGTTGTTGGGTGGTATTCAGCAGCGGAATCAGCCATTTGTTATCCAGACCTTTCTCGCGAGCCGCCTCTGCCGCCAGCGCAATCTCTTGCTCACTCATTCCTGCCAGCTGCGCGATATCGTTCACAACCAGACCGCCGGATTTATTTGCTGCCAGTAATCGCTGGTTAAACTGGCTGGTCAGGGTCGCAGCTTCTGTATTCAGTACTTTTAATTTTGCTTTATCAGCTTGCGCAAGTTTGGCTCCGGCAAGGACAAAACGTTGATGAATCACCTCCACCAGGCGGATGGATTCACTATCAAGCCCCAGGGATTCACGGCGCTGCCAGACAGCATCTACCCGCGCGAATAATTCACCGTTCAGATAGATATCATTAGCCAGTTCCGCCAGTTCAGCGGAAAACTGCTCGTCAAGACGCTGTAATTCATCATTGGTATGCGCCGCAGTCATCGCAAAAAAGACGCTGGTAACGCGGGTAAGTAATTCTCCGCTTTGTTCCAGTGCCAGAATAGTATTGTTGAAATCAGGCATTTGCGGGTTAAGCGCGATGGCAGCAATTTCTGCCCGCTTTTGCTGCATTCCCTCATCGAATGCCGGGCGATAGTGATGATTGGCAATTTGATCAAAATGGGGAGCCAGATACGGCAGTGTGCTTTGCACAAGGAAAGGATTCATTGTTGTCAT'
#'atggtagcccgtaaaggcgaagagctgttcactggtgtcgtccctattctggtggaactggatggtgatgtcaacggtcataagttttccgtgcgtggcgagggtgaaggtgacgcaactaatggtaaactgacgctgaagttcatctgtactactggtaaactgccggtaccttggccgactctggtaacgacgctgacttatggtgttcagtgctttgctcgttatccggaccatatgaagcagcatgacttcttcaagtccgccatgccggaaggctatgtgcaggaacgcacgatttcctttaaggatgacggcacgtacaaaacgcgtgcggaagtgaaatttgaaggcgataccctggtaaaccgcattgagctgaaaggcattgactttaaagaagatggcaatatcctgggccataagctggaatacaattttaacagccacaatgtttacatcaccgccgataaacaaaaaaatggcattaaagcgaattttaaaattcgccacaacgtggaggatggcagcgtgcagctggctgatcactaccagcaaaacactccaatcggtgatggtcctgttctgctgccagacaatcactatctgagcacgcaaagcgttctgtctaaagatccgaacgagaaacgcgatcatatggttctgctggagttcgtaaccgcagcgggcatcacgcatggtatggatgaactgtacaaaggttcgtaa'

In [ ]:
# Define length of PCR amplicon, and slice potential target area into chunks of that length for further analysis:

target_seq_moved_windows = moving_window(sfGFP_cds,90)
print(f"{len(target_seq_moved_windows)} potential amplicons identified!")

In [ ]:
fasta_string = []
for primer_idx, primer_pair_n in enumerate([(x[:20],x[-20:]) for x in target_seq_moved_windows]):
    fw_primer_sense, rv_primer_sense = primer_pair_n[0], primer_pair_n[1]
    if (Counter(primer_pair_n[0])['G']+Counter(primer_pair_n[0])['C'] == int(len(fw_primer_sense)/2)) and (Counter(primer_pair_n[1])['G']+Counter(primer_pair_n[1])['C'] == int(len(fw_primer_sense)/2)):
#         if (len(re.findall(r'A{4}',primer_pair[1])) > 0) or (len(re.findall(r'T{4}',primer_pair[1])) > 0) or (len(re.findall(r'G{4}',primer_pair[1])) > 0) or (len(re.findall(r'C{4}',primer_pair[1])) > 0):
#             pass
#         else:
        # 2- GC clamp
        end_3prime_fw, end_3prime_rv = primer_pair_n[0][-5:], primer_pair_n[1][:5]
        GC_count_in_3prim_end_fw, GC_count_in_3prim_end_rv = Counter(end_3prime_fw)['G']+Counter(end_3prime_fw)['C'], Counter(end_3prime_rv)['G']+Counter(end_3prime_rv)['C']
        if (GC_count_in_3prim_end_fw >1) & (GC_count_in_3prim_end_fw < 4) & (GC_count_in_3prim_end_rv >1) & (GC_count_in_3prim_end_rv < 4):
            fasta_string.append(f">primer_{primer_idx}_fw\n{primer_pair_n[0]}\n>primer_{primer_idx}_rv\n{primer_pair_n[1]}\n")
fasta_string = ''.join(fasta_string)
text_file = open("primer_search_cdp_CDS.fasta", "w")
n = text_file.write(fasta_string)
text_file.close()

In [ ]:
len(fasta_string.split('>primer'))

In [ ]:
primer_pair_n[1]#[::-1]

## Fully Factorial Substitution

In [ ]:
p = list(range(0,3))#+list(range(7,9))

x = 'AAA'

k = create_substitution_combinations(x,p)
print(k)
print('#:',len(k))

#print(list_to_CSV(k,'Test_substitution_4.csv'))


print(p)

In [ ]:
64*64

In [ ]:
dist_sorted = np.zeros((len(k),len(k)))
for i, seq1 in enumerate(k):
    for j, seq2 in enumerate(k):
        dist_sorted[i,j] = distance (seq1,seq2)
fig = plt.figure(figsize=(14,9))
plt.imshow(dist_sorted);

In [ ]:
np.where(dist_sorted==3)[0]


In [ ]:
"""First, choose the degree of dissimilarity between 3bp overhangs
"""
all_pairs_threes = np.where(dist_sorted==3)

three_pairs = []
three_pairs_df = []
for idx, column_index_value in enumerate(list(zip(np.where(dist_sorted==3)[0],np.where(dist_sorted==3)[1]))):
    #column_seq = k[column_value]
    #index_seq = k[all_pairs_twenty[1][idx]]
    
    three_pairs_df.append((k[column_index_value[0]], k[column_index_value[1]], 
          distance(k[column_index_value[0]], k[column_index_value[1]]))
         )
                                               
three_pairs_df = pd.DataFrame(three_pairs_df)

In [ ]:
three_pairs_df.loc[(three_pairs_df[0]=='AGC') & (three_pairs_df[1]=='GTT')]

In [ ]:
for x in three_pairs_df.loc[(three_pairs_df[0]=='AGC')][1]:
    display(three_pairs_df.loc[(three_pairs_df[0]=='GTT') & (three_pairs_df[1]==x)])